# 基于MindSpore实现强化学习

本实验主要专注于强化学习的原理，并基于MindSpore实现强化学习示例。

## 1、实验目的

- 掌握强化学习的原理。
- 掌握如何使用MindSpore实现强化学习。

## 2、强化学习原理介绍

（1）强化学习是一种机器学习的学习方式：

强化学习：基于环境的反馈而行动，通过不断与环境的交互、试错，最终完成特定目的或者使得整体行动收益最大化。强化学习不需要训练数据的label但是它需要每一步行动环境给予的反馈，是奖励还是惩罚，反馈可以量化，基于反馈不断调整训练对象的行为。

（2）强化学习的主要特点：试错学习；延迟反馈；时间是强化学习的一个重要因素；当前的行为影响后续接收到的数据。

（3）强化学习的基本组成部分：

- Agent(智能体)：强化学习训练的主体就是Agent：智能体。

- Environment(环境)：整个游戏的大背景就是环境。

- State(状态)：当前Environment和Agent所处的状态。Agent的位置在不断地变化，所以整个State处于变化中，State包含了Agent和Environment的状态。

- Action(行动)：基于当前的State、Agent可以采取哪些action，比如向左还是向右，向上还是向下，Action是和State强挂钩的。

- Reward(奖励)：Agent在当前State下，采取了某个特定的action后，会获得环境的一定反馈就是Reward。这里面用Reward进行统称，虽然Reward翻译成中文是“奖励”的意思，但其实强化学习中的Reward只是代表环境给予的“反馈”，可能是奖励也可能是惩罚。

- Q-Value(State,Action):Q-value是由State和Action组合在一起的决定的，这里的Value不是Reward，Reward是Value组成的一部分。实际的项目中我们会存储一张表——Q表，key是(state,action)，Value就是对应的Q-Value。每当agent进入到某个state下时我们就会来这张表进行查询，选择当前State下对应Value最大的Action，执行这个Action进入到下一个State，然后继续查表选择action，这样循环。Q-Value的价值就在于指导Agent在不同的State下选择Action。

（4）Q-learning算法的流程：
<img src="./Figures/fig001.jpg" style="zoom:70%" />

（5）Q-learning算法伪代码：

- 输入：${\alpha}$、${\gamma}$；

- 初始化：Q-Table(初始值全为0)；

   ${\quad}$${\quad}$${\quad}$S${\leftarrow}$state;a${\leftarrow}$action;

- 过程：

${\quad}$${\quad}$Agent随机选择一个状态S；
    
${\quad}$${\quad}$If S是Target S：

   ${\quad}$${\quad}$${\quad}$end；
     
${\quad}$${\quad}$else:

   ${\quad}$${\quad}$${\quad}$从当前状态S下，选择可以执行的a中对应Q(s,a)最大的a；
   
   ${\quad}$${\quad}$${\quad}$系统返回Reward；
   
   ${\quad}$${\quad}$${\quad}$更新当前的$Q(s,a){\leftarrow}Q(s,a) + {\alpha}[reward + {\gamma}Max_{a'}Q(s',a')-Q(s,a)]$；
   
   ${\quad}$${\quad}$${\quad}$更新状态S，继续循环；

## 3、实验环境

在动手进行实践之前，需要注意以下几点：
* 确保实验环境正确安装，包括安装MindSpore。安装过程：首先登录[MindSpore官网安装页面](https://www.mindspore.cn/install)，根据安装指南下载安装包及查询相关文档。同时，官网环境安装也可以按下表说明找到对应环境搭建文档链接，根据环境搭建手册配置对应的实验环境。
* 推荐使用交互式的计算环境Jupyter Notebook，其交互性强，易于可视化，适合频繁修改的数据分析实验环境。
* 实验也可以在华为云一站式的AI开发平台ModelArts上完成。
* 推荐实验环境：MindSpore版本=1.8；Python环境=3.7


|  硬件平台 |  操作系统  | 软件环境 | 开发环境 | 环境搭建链接 |
| :-----:| :----: | :----: |:----:   |:----:   |
| CPU | Windows-x64 | MindSpore1.8 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.1节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| GPU CUDA 10.1|Linux-x86_64| MindSpore1.8 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.2节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| Ascend 910  | Linux-x86_64| MindSpore1.8 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第四章](./MindSpore环境搭建实验手册.docx)|MindSpore官网华为开源自研AI框架MindSpore。自动微分、并行加持，一次训练，可多场景部署。支持端边云全场景的深度学习训练推理框架，主要应用于计算机视觉、自然语言处理等AI领域，面向数据科学家、算法工程师等人https://www.mindspore.cn/install

## 4、数据处理

### 4.1 数据准备

这里参考mofan大佬用到的一个treasure on right的弱智游戏来实现Q-Learning这个算法。

这个游戏就是宝藏在最右边的位置，然后训练一个智能体去获得这个宝藏。

各个参数的含义：

N_STATES：表示状态数量，其实也就是一维动作空间的位置数。

EPSILON：就是 ϵ-greedy 的ϵ。

同理，ALPHA，GAMMA也类似。

MAX_EPISODES：表示玩多少轮游戏来训练。

FRESH_TIME：是用来输出的参数，控制多久刷新一次页面。

TerminalFlag：用来记录游戏结束的标志符。

### 4.2 数据加载

**给出学习率、最大回合数等数据**

In [ ]:
N_STATES = 6                 #一维世界的宽度
ACTIONS = ["left", "right"]  # 可选动作选项
EPSILON = 0.9                # greedy贪婪度
ALPHA = 0.1                  # 学习率
GAMMA = 0.9                  # 未来奖励的折扣因子
MAX_EPISODES = 15            # 最大回合数
FRESH_TIME = 0.3             # 每回合移动间隔时间
TerminalFlag = "terminal"    # 目标终点

## 5、模型构建

**导入Python库&模块**

在使用前，导入需要的Python库。

In [ ]:
# 处理时间
import time
# 导入科学计算库
import numpy as np
# 导入数据分析处理库
import pandas as pd

**构建Q表并初始化**

In [ ]:
# 构建Q表，初始化Q表
def build_q_table(n_states, actions):
    return pd.DataFrame(
        np.zeros((n_states, len(actions))), # 初始置零
        columns=actions # q_table的index是所有对应的state(o所在的位置)，columns是对应的action(探险者选择left或者right)
    )

## 6、模型训练

**Q-learning算法训练**

行为选择函数、反馈函数、更新函数

In [ ]:
# 行为选择函数
def choose_action(state, q_table): # 当前状态s执行action的原则
    state_table = q_table.loc[state, :] # 选出这个state的所有 action的value值
    if (np.random.uniform() > EPSILON) or ((state_table == 0).all()): # 判断是否执行贪婪原则 or 当前状态还没有被探索过，即所有Q值为初始化值，没有被更新过
        action_name = np.random.choice(ACTIONS) # 随机选择动作
    else:
        action_name = state_table.idxmax() # 选择该状态下，Q值最大的动作执行
    return action_name # 返回动作名称

# 反馈函数，环境的反馈
def get_env_feedback(S, A):
    if A == "right":
        if S == N_STATES - 2: # 找到宝藏，注意位置从0开始计数
            S_, R = TerminalFlag, 1 # 到达终点，reward=1
        else:
            S_, R = S + 1, 0 # 否则向右移动，当前reward=0
    else:
        S_, R = max(0, S - 1), 0 # 左移或者已经在最左边不移动，reward=0
    return S_, R

# 更新函数
def update_env(S, episode, step_counter):
    env_list = ["-"] * (N_STATES - 1) + ["T"] # 构建环境的一维列表
    if S == TerminalFlag:
        interaction = 'Episode %s: total_steps = %s' % (episode + 1, step_counter)
        print(interaction)
        time.sleep(2) # 到达终点，输出当前一维列表的状态
    else:
        env_list[S] = '0' # 未到达终点，给所在位置标0
        interaction = ''.join(env_list)
        print(interaction)
        time.sleep(FRESH_TIME)

# 根据选择的行为更新q值
def rl():
    q_table = build_q_table(N_STATES, ACTIONS) # 引用Q表
    for episode in range(MAX_EPISODES):
        step_counter = 0
        S = 0
        is_terminated = False
        update_env(S, episode, step_counter)
        while not is_terminated: # 未到达终点下
            A = choose_action(S, q_table) # 动作选择
            S_, R = get_env_feedback(S, A) # 状态更新，reward反馈
            q_predict = q_table.loc[S, A] # 原来Q表里的初始化值为估计预测值

            if S_ != TerminalFlag: # 下一状态不为最终状态
                q_target = R + GAMMA * q_table.loc[S_, :].max() # 更新方程—Q-learning算法
            else:
                q_target = R # 下一状态为最终状态下的TD
                is_terminated = True
            q_table.loc[S, A] += ALPHA * (q_target - q_predict) # 更新Q表
            S = S_ # 跳转到下一个状态
            update_env(S, episode, step_counter + 1) 
            step_counter += 1
    return q_table # 返回更新后的Q表

## 7、模型预测

**通过函数调用，构建Q表**

In [1]:
# 主函数
if __name__ == '__main__':
    q_table = rl()
    print(q_table)

0----T
-0---T
0----T
-0---T
--0--T
---0-T
----0T
Episode 1: total_steps = 7
0----T
0----T
0----T
-0---T
--0--T
-0---T
0----T
0----T
-0---T
--0--T
---0-T
--0--T
-0---T
0----T
-0---T
0----T
-0---T
--0--T
---0-T
----0T
Episode 2: total_steps = 20
0----T
-0---T
0----T
0----T
-0---T
--0--T
---0-T
----0T
Episode 3: total_steps = 8
0----T
0----T
0----T
-0---T
--0--T
---0-T
----0T
Episode 4: total_steps = 7
0----T
0----T
-0---T
--0--T
---0-T
----0T
Episode 5: total_steps = 6
0----T
0----T
-0---T
--0--T
---0-T
----0T
Episode 6: total_steps = 6
0----T
-0---T
--0--T
-0---T
--0--T
---0-T
----0T
Episode 7: total_steps = 7
0----T
-0---T
--0--T
---0-T
----0T
---0-T
----0T
Episode 8: total_steps = 7
0----T
-0---T
--0--T
---0-T
----0T
Episode 9: total_steps = 5
0----T
0----T
-0---T
--0--T
---0-T
----0T
Episode 10: total_steps = 6
0----T
-0---T
--0--T
---0-T
----0T
Episode 11: total_steps = 5
0----T
-0---T
--0--T
---0-T
----0T
Episode 12: total_steps = 5
0----T
-0---T
--0--T
---0-T
----0T
Episode 13: to